## Notebook para obetener el valor total de la compra

Comenzamos importanto la base, y utilizando aquellas imágenes que SI son facturas:

In [ ]:
# Loads Constants, imports and utilitary functions
%run 0_Utils.ipynb
df = getLatestDataset()
df_facturas = df[df['doc_type'] == 'invoice']
df_facturas.head()

In [86]:
df_facturas.shape

(1226, 9)

Creamos la función de extración del valor total de la compra con textract:

In [87]:
def getInvoiceTotalWithTextract(filename):
    client = boto3.client('textract', region_name=REGION_NAME)
    response = client.analyze_expense(Document={'S3Object': {'Bucket': BUCKET, 'Name': filename}})
    
    if len(response['ExpenseDocuments']) == 0:
        pass
    else:
        for summary_field in response['ExpenseDocuments'][0]['SummaryFields']:
            if "Type" in summary_field and summary_field['Type']['Text'] == 'TOTAL':
                #print((summary_field['ValueDetection']['Text']),(filename))
                return (summary_field['ValueDetection']['Text'])


Validamos con alguna factura el resultado:

In [88]:
print(getInvoiceTotalWithTextract('raw_images/3d73e2f4087001d066669be4d48e6b9e.jpg'))


COP 14000 80


Corremos la función de extracción por todos los registros de la base:

In [89]:
#df_facturas= df_facturas.iloc[150:155] # Primeras cinco filas
df_facturas['total_extraido']=df_facturas['filename'].map(getInvoiceTotalWithTextract)
df_facturas.head()

/tmp/ipykernel_7278/2091837015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facturas['total_extraido']=df_facturas['filename'].map(getInvoiceTotalWithTextract)


,id,filename,sharpness,doc_type,issuer,total_value,items,text,Categoria,total_extraido
0,a154b5b134d6d2f31909f8e808c29bae,raw_images/a154b5b134d6d2f31909f8e808c29bae.jpg,0,invoice,CREPES Y WAFFLES,"$19,800",Cono 2 sab crepes&waffles,CREPES Y WAFFLES S.A. AV 19 860.076.919-1 CIIU...,Alimentación,None
2,4b589ece128d136570273e078540b696,raw_images/4b589ece128d136570273e078540b696.jpg,0,invoice,TIENDAS ARA,"$4,585",Mandarina x,TIENDAS ARA Jeronimo Martins Colombia S.A.S. N...,Grande superficie,4.585
3,15d39e755caee068c6fc5936a77dac95,raw_images/15d39e755caee068c6fc5936a77dac95.jpg,1,invoice,MUNDO VERDE,"$75,600","Waffle mix, wrap aguacate, wok veggie ori",MUNDO VERDE 8475 SALUD GOURMET S.A.S NIT: 9002...,Alimentación,75.600
4,19ef3de474df59454abadc705ff39174,raw_images/19ef3de474df59454abadc705ff39174.jpg,0,invoice,DROGAS LA FRAGUA,"$8,300","Gaviscon doble accion susp, gaviscon doble ac...",(+ Sfarma / DROGAS LA FRAGUA B.A.S NIT 8300008...,Salud y Bienestar,"8,108"
5,3d73e2f4087001d066669be4d48e6b9e,raw_images/3d73e2f4087001d066669be4d48e6b9e.jpg,0,invoice,DOLLARCITY,"$14,000",Pinzas led en forma de corazon,Dollarcity DOLLARCITY PLAZA DE LAS S.A.S. AMER...,Grande superficie,COP 14000 80


Como vemos que en el total extraído hay un comun de donde alguns textos incluyen "total", "cop" y "$".

Entonces vamos a eliminarlos, y de la columna total tambien vamos a eliminarlos:

In [90]:
def reemplazar_campos(file):
    reemplazo = file.replace('$', '')
    reemplazo = reemplazo.replace('TOTAL:', '')
    reemplazo = reemplazo.replace('COP', '')
    reemplazo = reemplazo.strip(' ')
    return reemplazo

df_facturas['total_extraido']=df_facturas['total_extraido'].fillna("0")
df_facturas['total_value']=df_facturas['total_value'].fillna("0")
df_facturas['total_extraido'] = df_facturas['total_extraido'].map(reemplazar_campos)
df_facturas['total_value'] = df_facturas['total_value'].map(reemplazar_campos)

df_facturas.head()


/tmp/ipykernel_7278/1523821592.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facturas['total_extraido']=df_facturas['total_extraido'].fillna("0")
/tmp/ipykernel_7278/1523821592.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_facturas['total_value']=df_facturas['total_value'].fillna("0")
/tmp/ipykernel_7278/1523821592.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

,id,filename,sharpness,doc_type,issuer,total_value,items,text,Categoria,total_extraido
0,a154b5b134d6d2f31909f8e808c29bae,raw_images/a154b5b134d6d2f31909f8e808c29bae.jpg,0,invoice,CREPES Y WAFFLES,"19,800",Cono 2 sab crepes&waffles,CREPES Y WAFFLES S.A. AV 19 860.076.919-1 CIIU...,Alimentación,0
2,4b589ece128d136570273e078540b696,raw_images/4b589ece128d136570273e078540b696.jpg,0,invoice,TIENDAS ARA,"4,585",Mandarina x,TIENDAS ARA Jeronimo Martins Colombia S.A.S. N...,Grande superficie,4.585
3,15d39e755caee068c6fc5936a77dac95,raw_images/15d39e755caee068c6fc5936a77dac95.jpg,1,invoice,MUNDO VERDE,"75,600","Waffle mix, wrap aguacate, wok veggie ori",MUNDO VERDE 8475 SALUD GOURMET S.A.S NIT: 9002...,Alimentación,75.600
4,19ef3de474df59454abadc705ff39174,raw_images/19ef3de474df59454abadc705ff39174.jpg,0,invoice,DROGAS LA FRAGUA,"8,300","Gaviscon doble accion susp, gaviscon doble ac...",(+ Sfarma / DROGAS LA FRAGUA B.A.S NIT 8300008...,Salud y Bienestar,"8,108"
5,3d73e2f4087001d066669be4d48e6b9e,raw_images/3d73e2f4087001d066669be4d48e6b9e.jpg,0,invoice,DOLLARCITY,"14,000",Pinzas led en forma de corazon,Dollarcity DOLLARCITY PLAZA DE LAS S.A.S. AMER...,Grande superficie,14000 80


Vemos que hay algunos campos en donde no fue posible realizar una extarcción del valor total de la factura, vamos a validar cuantos campos son:

In [91]:
df_facturas.apply(lambda x: x['total_extraido'] == '0', axis=1).sum()

343

In [92]:
!pip install thefuzz
from thefuzz import fuzz

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/tensorflow2_p38/bin/python -m pip install --upgrade pip' command.


In [93]:
def comparacion_fuzzy(valor_factura,texto_extraido):
    print(fuzz.ratio(valor_factura,texto_extraido))

comparacion_fuzzy("LARAU","LAURAU S")

77


In [94]:
#f_facturas['diferencia_fuzzy'] = df_facturas.apply(lambda x: fuzz.ratio(df_facturas['total_value'],df_facturas['total_extraido']),axis = 1)
#df_facturas['diferencia_fuzzy']=df_facturas['diferencia_fuzzy'].map(comparacion_fuzzy,df_facturas['total_extraido'])
#f_facturas.head()

In [95]:
def comparacion_fuzzy(valor_factura,texto_extraido):
    ratio = fuzz.ratio(valor_factura,texto_extraido)
    token_sort_ratio = fuzz.token_sort_ratio(valor_factura,texto_extraido)
    return ratio,token_sort_ratio

df_facturas[['Diferencia_fuzzy_ratio', 'Diferencia_fuzzy_sort_token']]=df_facturas.apply(lambda x: comparacion_fuzzy(x['total_value'], x['total_extraido']),axis=1,result_type='expand')

df_facturas.head()

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,id,filename,sharpness,doc_type,issuer,total_value,items,text,Categoria,total_extraido,Diferencia_fuzzy_ratio,Diferencia_fuzzy_sort_token
0,a154b5b134d6d2f31909f8e808c29bae,raw_images/a154b5b134d6d2f31909f8e808c29bae.jpg,0,invoice,CREPES Y WAFFLES,"19,800",Cono 2 sab crepes&waffles,CREPES Y WAFFLES S.A. AV 19 860.076.919-1 CIIU...,Alimentación,0,29,29
2,4b589ece128d136570273e078540b696,raw_images/4b589ece128d136570273e078540b696.jpg,0,invoice,TIENDAS ARA,"4,585",Mandarina x,TIENDAS ARA Jeronimo Martins Colombia S.A.S. N...,Grande superficie,4.585,80,100
3,15d39e755caee068c6fc5936a77dac95,raw_images/15d39e755caee068c6fc5936a77dac95.jpg,1,invoice,MUNDO VERDE,"75,600","Waffle mix, wrap aguacate, wok veggie ori",MUNDO VERDE 8475 SALUD GOURMET S.A.S NIT: 9002...,Alimentación,75.600,83,100
4,19ef3de474df59454abadc705ff39174,raw_images/19ef3de474df59454abadc705ff39174.jpg,0,invoice,DROGAS LA FRAGUA,"8,300","Gaviscon doble accion susp, gaviscon doble ac...",(+ Sfarma / DROGAS LA FRAGUA B.A.S NIT 8300008...,Salud y Bienestar,"8,108",60,60
5,3d73e2f4087001d066669be4d48e6b9e,raw_images/3d73e2f4087001d066669be4d48e6b9e.jpg,0,invoice,DOLLARCITY,"14,000",Pinzas led en forma de corazon,Dollarcity DOLLARCITY PLAZA DE LAS S.A.S. AMER...,Grande superficie,14000 80,71,57


In [96]:
df_facturas['Diferencia_fuzzy_ratio'].mean()

63.96982055464927

In [97]:
df_facturas['Diferencia_fuzzy_sort_token'].mean()

68.58646003262643

Ahora vamos a implementarl la diferencia de Fuzzy para aquellos campos en donde el textract si logró realizar una extracción del valor total de la factua

In [98]:
df_facturas_extraido = df_facturas[df_facturas['total_extraido'] != '0']
df_facturas_extraido.head()

,id,filename,sharpness,doc_type,issuer,total_value,items,text,Categoria,total_extraido,Diferencia_fuzzy_ratio,Diferencia_fuzzy_sort_token
2,4b589ece128d136570273e078540b696,raw_images/4b589ece128d136570273e078540b696.jpg,0,invoice,TIENDAS ARA,"4,585",Mandarina x,TIENDAS ARA Jeronimo Martins Colombia S.A.S. N...,Grande superficie,4.585,80,100
3,15d39e755caee068c6fc5936a77dac95,raw_images/15d39e755caee068c6fc5936a77dac95.jpg,1,invoice,MUNDO VERDE,"75,600","Waffle mix, wrap aguacate, wok veggie ori",MUNDO VERDE 8475 SALUD GOURMET S.A.S NIT: 9002...,Alimentación,75.600,83,100
4,19ef3de474df59454abadc705ff39174,raw_images/19ef3de474df59454abadc705ff39174.jpg,0,invoice,DROGAS LA FRAGUA,"8,300","Gaviscon doble accion susp, gaviscon doble ac...",(+ Sfarma / DROGAS LA FRAGUA B.A.S NIT 8300008...,Salud y Bienestar,"8,108",60,60
5,3d73e2f4087001d066669be4d48e6b9e,raw_images/3d73e2f4087001d066669be4d48e6b9e.jpg,0,invoice,DOLLARCITY,"14,000",Pinzas led en forma de corazon,Dollarcity DOLLARCITY PLAZA DE LAS S.A.S. AMER...,Grande superficie,14000 80,71,57
7,134cada3d02d8b379afbfc293f3abeeb,raw_images/134cada3d02d8b379afbfc293f3abeeb.jpg,1,invoice,SOMOS MASA,"73,800","8 pain au Chocolat, mini brownies lata, bolsa ...",SOMOS MASA S.A.S NIT: 900.432.416-9 SUCURSAL A...,Alimentación,"73,800",100,100


In [99]:
df_facturas_extraido[['Diferencia_fuzzy_ratio', 'Diferencia_fuzzy_sort_token']]=df_facturas_extraido.apply(lambda x: comparacion_fuzzy(x['total_value'], x['total_extraido']),axis=1,result_type='expand')

df_facturas_extraido.head()

/home/ec2-user/anaconda3/envs/tensorflow2_p38/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,id,filename,sharpness,doc_type,issuer,total_value,items,text,Categoria,total_extraido,Diferencia_fuzzy_ratio,Diferencia_fuzzy_sort_token
2,4b589ece128d136570273e078540b696,raw_images/4b589ece128d136570273e078540b696.jpg,0,invoice,TIENDAS ARA,"4,585",Mandarina x,TIENDAS ARA Jeronimo Martins Colombia S.A.S. N...,Grande superficie,4.585,80,100
3,15d39e755caee068c6fc5936a77dac95,raw_images/15d39e755caee068c6fc5936a77dac95.jpg,1,invoice,MUNDO VERDE,"75,600","Waffle mix, wrap aguacate, wok veggie ori",MUNDO VERDE 8475 SALUD GOURMET S.A.S NIT: 9002...,Alimentación,75.600,83,100
4,19ef3de474df59454abadc705ff39174,raw_images/19ef3de474df59454abadc705ff39174.jpg,0,invoice,DROGAS LA FRAGUA,"8,300","Gaviscon doble accion susp, gaviscon doble ac...",(+ Sfarma / DROGAS LA FRAGUA B.A.S NIT 8300008...,Salud y Bienestar,"8,108",60,60
5,3d73e2f4087001d066669be4d48e6b9e,raw_images/3d73e2f4087001d066669be4d48e6b9e.jpg,0,invoice,DOLLARCITY,"14,000",Pinzas led en forma de corazon,Dollarcity DOLLARCITY PLAZA DE LAS S.A.S. AMER...,Grande superficie,14000 80,71,57
7,134cada3d02d8b379afbfc293f3abeeb,raw_images/134cada3d02d8b379afbfc293f3abeeb.jpg,1,invoice,SOMOS MASA,"73,800","8 pain au Chocolat, mini brownies lata, bolsa ...",SOMOS MASA S.A.S NIT: 900.432.416-9 SUCURSAL A...,Alimentación,"73,800",100,100


In [100]:
df_facturas_extraido['Diferencia_fuzzy_ratio'].mean()

79.480181200453

In [101]:
df_facturas_extraido['Diferencia_fuzzy_sort_token'].mean()

85.89014722536807